In [ ]:
# LDA(Latent Dirichlet Allocation)
## 토픽은 각각 단어 분포를 가지고 있다고 가정한다. 
### 1. n개의 토픽을 지정한다. 
### 2. 무작위로 토픽의 혼합 비율을 설정
### 3. 단어들을 토픽에 할당
####### 현재 문서의 모든 단어를 토픽에 할당
####### 현재 문서에서 토픽 비율 업데이트
####### 모든 문서에서 토픽 비율 업데이트
### 4. 반복

# 1. 데이터 불러오기

In [ ]:
import pandas as pd

df = pd.read_csv(
    "./data/appreply2.csv",
    index_col=0
)
df.head()

# 2. 텍스트->숫자

## 1) 전처리

In [ ]:
# 목표
# 나는 밥을 먹었다 -> (토크나이징) 나 / 는 / 밥 / 을 / 먹었다. -> 나 는 밥 을 먹었다.
# 단어들이 띄어쓰기를 구분자로 하여 하나의 문자열로 완성되어야 한다.

In [ ]:
# Kiwipiepy 품사 태그 목록 https://bab2min.github.io/kiwipiepy/v0.22.2/kr/#_11

# STEP1. 전처리한 문장을 담을 빈리스트를 준비한다. sent_list 
# STEP2. df["text"]에서 문장을 하나씩 뺀다. sent 
# STEP3. 문장을 정규표현식을 이용해서 전처리한다. clean_sent
# STEP4. clean_sent를 토크나이징한다. (Kiwi 형태소 분석기 사용) result
         # 1) 조건 : 조사(J), 어미(E), 접미사(X)는 포함하지 않는다.
         # 2) 조건 : 한 글자인 단어는 포함하지 않는다.
# STEP5. result를 " "를 구분자로 하나의 문자열로 합친다. result_str
# STEP6. result_str을 sent_list에 넣는다.

In [ ]:
import re 
from kiwipiepy import Kiwi 

kiwi = Kiwi()

In [ ]:
sample_test = kiwi.tokenize("안녕하세요. 저는 형태소 분석기에요.")
# print(sample_test)
for x in sample_test:
    print(f"x = {x}")
    print(f"word = {x.form}, pos = {x.tag}")

In [ ]:
# STEP1. 전처리한 문장을 담을 빈리스트를 준비한다. sent_list 
sent_list = []

# STEP2. df["text"]에서 문장을 하나씩 뺀다. sent 
for sent in df["text"]:
    # STEP3. 문장을 정규표현식을 이용해서 전처리한다. clean_sent
    print("STEP3. 문장을 전처리합니다.")
    clean_sent = re.sub("[^a-zA-Z가-힣\s]", "", sent)
    # STEP4. clean_sent를 토크나이징한다. (Kiwi 형태소 분석기 사용) result
    print("STEP4. 문장을 토크나이징합니다.")
    result = kiwi.tokenize(clean_sent)
    print("\tsub_list를 생성합니다.")
    sub_list = []
    print("\t탐색을 시작합니다.")
    for x in result:
        # 1) 조건 : 조사(J), 어미(E), 접미사(X)는 포함하지 않는다. -> pos[0] in ["J", "E", "X"] 건너뛰기
        word = x.form
        pos = x.tag 
        if pos[0] in ["J", "E", "X"]:
            # print(f"\t건너뛰기!! word={word} pos={pos}")
            continue
        # 2) 조건 : 한 글자인 단어는 포함하지 않는다.
        if len(word) > 1:
            sub_list.append(word)
        # print(f"\t추가!! word={word} pos={pos}")
    print("\t탐색을 종료합니다.")
    print(f"[SUB LIST] {sub_list}")
    # STEP5. result를 " "를 구분자로 하나의 문자열로 합친다. result_str
    result_str = " ".join(sub_list)
    print(f"[RESULT_STR] {result_str}")
    # STEP6. result_str을 sent_list에 넣는다.
    sent_list.append(result_str)
    print(sent)
    print("="*50)

In [ ]:
print(len(sent_list))
for old, new in zip(df["text"].tolist(), sent_list):
    print(f"[OLD] {old}")
    print(f"[NEW] {new}")
    print("="*100)

## 2) 수치화

In [ ]:
# 라이브러리 설치
# uv add scikit-learn

In [ ]:
# CountVectorizer란?
# 문장에 해당 단어가 몇 개 들어가 있나요?
#      단어1  단어2  단어3 ...
# 문장1
# 문장2 
# ...

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(
    max_df=0.1,         # 전체 단어의 등장 비율이 p이상인 것만 사용
    min_df=2,           # 이 단어가 적어도 n개 이상 있는 것만 사용
    max_features=1000,  # 최대 몇 개까지 나타낼 것인가
    ngram_range=(1,2)   # 단어의 조합 설정(ex. 1개만 사용)
)
feat_vec = count_vectorizer.fit_transform(sent_list)
# 문장 -> 스페이스 기준으로 쪼갠다.  -> 단어들이 나열
# 각 문장마다 단어가 몇번 들어가 있는지 카운트를 세고
# 하나의 매트릭스로 표현한다. 

In [ ]:
feat_vec.shape

In [ ]:
import pandas as pd

# 1. Sparse Matrix를 일반 배열(Dense)로 변환: .toarray()
# 2. 열 이름(단어 목록) 가져오기: get_feature_names_out()
df_vec = pd.DataFrame(
    feat_vec.toarray(), 
    columns=count_vectorizer.get_feature_names_out()
)

# 결과 확인
df_vec.head()

# 3. 토픽분석 시각화

In [ ]:
# 라이브러리 설치
# uv add pyLDAvis

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5) # 몇 개의 topic으로 할까요?
lda.fit(feat_vec)

In [ ]:
import pyLDAvis.lda_model 

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vec, count_vectorizer)
pyLDAvis.display(vis)

In [ ]:
# 행: 문장 
# 열: 그룹(n_component)
# 값: 문장이 그룹에 포함될 확률
# ex. 문장1  [0.1  0.8  0.1]  -> 이 문장1은 그룹 2이다. 
sent_topic = lda.transform(feat_vec)
print(sent_topic)

In [ ]:
import numpy as np 

sent1 = np.array([0.01300719, 0.976353, 0.01063981])
print(sent1.sum())
print(sent1.max())        # 배열의 최댓값
print(sent1.argmax())     # 배열에 최댓값이 있는 위치

In [ ]:
# 반복문
# STEP0. 빈 리스트 2개를 준비한다. - topic_idx_list, topic_prob_list
# STEP1. sent_topic에서 요소 하나하나씩 뽑는다. [0.01300719, 0.976353, 0.01063981]
# STEP2. 이 문장을 어떤 그룹으로 분류할 건지(argmax()) - topic_idx
# STEP3. 이 문장이 분류되었을 때 그 확률(max()) - topic_prob
# STEP4. topic_idx, topic_prob을 topic_idx_list, topic_prob_list에 담는다.

In [ ]:
# STEP0. 빈 리스트 2개를 준비한다. - topic_idx_list, topic_prob_list
topic_idx_list = []
topic_prob_list = []

# STEP1. sent_topic에서 요소 하나하나씩 뽑는다. [0.01300719, 0.976353, 0.01063981]
for sent in sent_topic:
    # STEP2. 이 문장을 어떤 그룹으로 분류할 건지(argmax()) - topic_idx
    topic_idx = sent.argmax()

    # STEP3. 이 문장이 분류되었을 때 그 확률(max()) - topic_prob
    topic_prob = sent.max()

    # STEP4. topic_idx, topic_prob을 topic_idx_list, topic_prob_list에 담는다.
    topic_idx_list.append(topic_idx)
    topic_prob_list.append(topic_prob)
    print(sent)
    print(f"{topic_idx} 위치에 최댓값 {topic_prob:.4f}이 있습니다.")
    print("="*50)

In [ ]:
# 새로운 열 생성
df["topic_no"] = topic_idx_list 
df["topic_prob"] = topic_prob_list

In [ ]:
df.head()

In [ ]:
# 어떤 토픽이 가장 많이 있나요? - value_counts()
df["topic_no"].value_counts()

In [ ]:
# 각 토픽별로 확률이 가장 높은 순서대로 보여주세요(셀 3개, 토픽 0, 1, 2각각 프린트) - sort.values()
## 토픽 0 인 데이터 조회하기
df.loc[ df["topic_no"]==0 , :].sort_values(by=["topic_prob"], ascending=False)

In [ ]:
## 토픽 1 인 데이터 조회하기
df.loc[ df["topic_no"]==1 , :].sort_values(by=["topic_prob"], ascending=False)

In [ ]:
## 토픽 2 인 데이터 조회하기
df.loc[ df["topic_no"]==2 , :].sort_values(by=["topic_prob"], ascending=False)

In [ ]:
# 목적: 토픽별로 주제를 정의한다. 
# 각 토픽별로 확률이 가장 높은 5개 문장을 한번에 프린트해주세요. - 반복문
n_topic = df["topic_no"].nunique() # topic_no는 몇 개의 그룹인가요?
print(f"{n_topic}개의 그룹이 있습니다.")

for n in range(n_topic):
    print(f"# TOPIC {n}")
    # df에서 topic이 n인 데이터만 추출한 후, topic_prob 기준으로 내림차순 
    topic_df = df.loc[ df["topic_no"]==n , :].sort_values(by=["topic_prob"], ascending=False)
    # topic_df의 text를 가져온 후 맨 위에서 5개만 출력
    for i in range(3):
        print(topic_df["text"].values[i])
        print("-"*100)
    print("="*100)